<a href="https://colab.research.google.com/github/spinafex/crispy-happiness/blob/main/brn_strat1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import itertools
from statsmodels.tsa.stattools import coint
from scipy.stats import zscore
import statsmodels.api as sm
from scipy.optimize import minimize
import warnings

In [56]:
# Load data from a CSV file
def load_data(file_path):
    data = pd.read_csv(file_path, index_col=0, parse_dates=True, dayfirst=True)
    data = data.interpolate(method='linear', axis=0, limit_direction='both', limit=5)

    return data

# Split data into training and testing sets
def split_data(data, split_ratio=0.5):
    split_point = int(len(data) * split_ratio)
    train_data = data[:split_point]
    test_data = data[split_point:]
    #print(train_data) #debugger
    return train_data, test_data

# Calculate daily returns for the given data
def calculate_daily_returns(data):
    # Convert the data to a NumPy array
    data_array = data.values

    # Check if the first row contains labels or numeric data
    if data_array.dtype.kind == 'O':
        # First row contains labels
        labels = data_array[0].tolist()
        data_array = data_array[1:]
    else:
        # First row contains numeric data
        labels = data.columns.tolist()
        #print(labels) #debug

    # Calculate the daily returns
    pct_change = [(x - y) / y for x, y in zip(data_array[1:], data_array[:-1])]
    pct_change = np.array(pct_change)

    return labels, pct_change

def find_cointegrated_pairs(labels, pct_change):
    # Convert pct_change to a NumPy array if necessary
    if not isinstance(pct_change, np.ndarray):
        pct_change = np.array(pct_change)

    # Drop rows with NaN or inf values
    pct_change = pct_change[~np.isnan(pct_change).any(axis=1)]
    pct_change = pct_change[~np.isinf(pct_change).any(axis=1)]

    cointegrated_pairs = []

    # Iterate over each pair of assets
    for i in range(pct_change.shape[1]):
        for j in range(i + 1, pct_change.shape[1]):
            asset1 = pct_change[:, i]
            asset2 = pct_change[:, j]

            # Test for cointegration
            score, pvalue, _ = coint(asset1, asset2)

            # Check if the pair is cointegrated
            if pvalue < 0.05:
                cointegrated_pairs.append((labels[i], labels[j], pvalue))

    return cointegrated_pairs

file_path = 'crypto.csv'
data = load_data(file_path)
train_data, _ = split_data(data, split_ratio=0.5)
labels, pct_change = calculate_daily_returns(train_data)
cointegrated_pairs = find_cointegrated_pairs(labels, pct_change)
print(cointegrated_pairs)

[('BTC', 'ETH', 6.7886492716240616e-09), ('BTC', 'DOGE', 1.469532910382539e-09), ('BTC', 'ADA ', 1.506304231032239e-25), ('BTC', 'SHIB', 6.820111003321325e-09), ('BTC', 'AVAX', 3.3500908304090666e-14), ('BTC', 'LINK', 7.39353263321604e-29), ('BTC', 'TRX', 6.794200675970872e-14), ('BTC', 'UNI', 5.350516600090906e-09), ('BTC', 'MATIC', 1.0204408377940847e-06), ('ETH', 'DOGE', 3.373856812438888e-11), ('ETH', 'ADA ', 1.4090152874910037e-25), ('ETH', 'SHIB', 7.10292868411349e-25), ('ETH', 'AVAX', 1.518570828200927e-11), ('ETH', 'LINK', 2.1457965400527525e-25), ('ETH', 'TRX', 2.2122839496054053e-29), ('ETH', 'UNI', 8.694383397478782e-25), ('ETH', 'MATIC', 1.1891945421798322e-06), ('DOGE', 'ADA ', 1.2284616609566376e-18), ('DOGE', 'SHIB', 1.216780274347241e-16), ('DOGE', 'AVAX', 1.7687683535111327e-17), ('DOGE', 'LINK', 6.076406978866814e-18), ('DOGE', 'TRX', 2.373433289950151e-17), ('DOGE', 'UNI', 2.5654201073693612e-17), ('DOGE', 'MATIC', 1.204275070566345e-17), ('ADA ', 'SHIB', 2.080942563

In [103]:
import numpy as np
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.stattools import adfuller
import pandas as pd

# Load data from a CSV file
def load_data(file_path):
    data = pd.read_csv(file_path, index_col=0, parse_dates=True, dayfirst=True)
    data = data.interpolate(method='linear', axis=0, limit_direction='both', limit=5)

    return data

# Split data into training and testing sets
def split_data(data, split_ratio=0.5):
    split_point = int(len(data) * split_ratio)
    train_data = data[:split_point]
    test_data = data[split_point:]
    #print(train_data) #debugger
    return train_data, test_data

# Calculate daily returns for the given data
def calculate_daily_returns(data):
    # Convert the data to a NumPy array
    data_array = data.values

    # Check if the first row contains labels or numeric data
    if data_array.dtype.kind == 'O':
        # First row contains labels
        labels = data_array[0].tolist()
        data_array = data_array[1:]
    else:
        # First row contains numeric data
        labels = data.columns.tolist()
        #print(labels) #debug

    # Calculate the daily returns
    pct_change = [(x - y) / y for x, y in zip(data_array[1:], data_array[:-1])]
    pct_change = np.array(pct_change)

    return labels, pct_change

import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller, coint

def find_cointegrated_pairs(labels, pct_change):
    # Convert pct_change to NumPy array if necessary
    if not isinstance(pct_change, np.ndarray):
        pct_change = np.array(pct_change)

    # Drop rows with NaN or inf values
    pct_change = pct_change[~np.isnan(pct_change).any(axis=1)]
    pct_change = pct_change[~np.isinf(pct_change).any(axis=1)]

    n = pct_change.shape[1]
    corr_matrix = np.zeros((n, n))
    p_value_matrix = np.ones((n, n))
    np.fill_diagonal(p_value_matrix, np.nan)

    stationarity_results = []

    for i in range(n):
        for j in range(i + 1, n):
            asset1 = pct_change[:, i]
            asset2 = pct_change[:, j]

            # Calculate correlation coefficient
            corr_coef = np.corrcoef(asset1, asset2)[0, 1]

            # Test for stationarity
            adf_stat, p_val, _, _, _, _ = adfuller(asset1 - asset2)
            is_stationary = p_val < 0.05

            # Test for cointegration
            _, coint_pvalue, _ = coint(asset1, asset2)

            corr_matrix[i, j] = corr_coef
            corr_matrix[j, i] = corr_coef
            p_value_matrix[i, j] = coint_pvalue
            p_value_matrix[j, i] = coint_pvalue

            # Store the stationarity test result
            pair_label = f"{labels[i]} - {labels[j]}"
            result = {
                'Pair': pair_label,
                'ADF Statistic': adf_stat,
                'p-value': p_val,
                'Is Stationary': is_stationary
            }
            stationarity_results.append(result)

    corr_df = pd.DataFrame(corr_matrix, columns=labels, index=labels)
    p_value_df = pd.DataFrame(p_value_matrix, columns=labels, index=labels)
    stationarity_results_df = pd.DataFrame(stationarity_results)

    return corr_df, p_value_df, stationarity_results_df


    corr_df = pd.DataFrame(corr_matrix, columns=labels, index=labels)
    p_value_df = pd.DataFrame(p_value_matrix, columns=labels, index=labels)
    results = [list(result) for result in results]
    results = pd.DataFrame(results, columns=labels, index=labels)

    return corr_df, p_value_df, results



file_path = 'crypto.csv'
data = load_data(file_path)
train_data, _ = split_data(data, split_ratio=0.5)
labels, pct_change = calculate_daily_returns(train_data)
corr_df, p_value_df, results = find_cointegrated_pairs(labels, pct_change)
#print(corr_df)
#corr_df.to_csv('corr_matrix', index=True, header=True)
#p_value_df.to_csv('pvalue_matrix', index=True, header=True)
results.to_csv('adf', index=True, header=True)

In [71]:
import numpy as np
from statsmodels.tsa.stattools import coint
import pandas as pd

# Load data from a CSV file
def load_data(file_path):
    data = pd.read_csv(file_path, index_col=0, parse_dates=True, dayfirst=True)
    data = data.interpolate(method='linear', axis=0, limit_direction='both', limit=5)

    return data

# Split data into training and testing sets
def split_data(data, split_ratio=0.5):
    split_point = int(len(data) * split_ratio)
    train_data = data[:split_point]
    test_data = data[split_point:]
    #print(train_data) #debugger
    return train_data, test_data

# Calculate daily returns for the given data
def calculate_daily_returns(data):
    # Convert the data to a NumPy array
    data_array = data.values

    # Check if the first row contains labels or numeric data
    if data_array.dtype.kind == 'O':
        # First row contains labels
        labels = data_array[0].tolist()
        data_array = data_array[1:]
    else:
        # First row contains numeric data
        labels = data.columns.tolist()
        #print(labels) #debug

    # Calculate the daily returns
    pct_change = [(x - y) / y for x, y in zip(data_array[1:], data_array[:-1])]
    pct_change = np.array(pct_change)

    return labels, pct_change

# Find cointegrated pairs
def find_cointegrated_pairs(train_data):
    returns = calculate_daily_returns(train_data)
    n = returns.shape[1]
    keys = returns.columns
    pairs = []
    pvalues = np.ones((n, n))
    for i, j in itertools.combinations(range(n), 2):
        S1 = returns[keys[i]]
        S2 = returns[keys[j]]
        result = coint(S1, S2)
        pvalue = result[1]
        pvalues[i, j] = pvalue
        if pvalue < 0.05:
            pairs.append((keys[i], keys[j], pvalue))
    return pairs, pvalues

In [66]:
!pip install colorama

In [ ]:
import numpy as np
from statsmodels.tsa.stattools import coint
import pandas as pd

# Load data from a CSV file
def load_data(file_path):
    data = pd.read_csv(file_path, index_col=0, parse_dates=True, dayfirst=True)
    data = data.interpolate(method='linear', axis=0, limit_direction='both', limit=5)

    return data

# Split data into training and testing sets
def split_data(data, split_ratio=0.5):
    split_point = int(len(data) * split_ratio)
    train_data = data[:split_point]
    test_data = data[split_point:]
    #print(train_data) #debugger
    return train_data, test_data

# Calculate daily returns for the given data
def calculate_daily_returns(data):
    # Convert the data to a NumPy array
    data_array = data.values

    # Check if the first row contains labels or numeric data
    if data_array.dtype.kind == 'O':
        # First row contains labels
        labels = data_array[0].tolist()
        data_array = data_array[1:]
    else:
        # First row contains numeric data
        labels = data.columns.tolist()
        #print(labels) #debug

    # Calculate the daily returns
    pct_change = [(x - y) / y for x, y in zip(data_array[1:], data_array[:-1])]
    pct_change = np.array(pct_change)

    return labels, pct_change

def find_cointegrated_pairs(labels, pct_change):
    print("Entering find_cointegrated_pairs function")

    # Convert pct_change to NumPy array if necessary
    if not isinstance(pct_change, np.ndarray):
        pct_change = np.array(pct_change)
        print("Converted pct_change to NumPy array")
    else:
        print("pct_change is already a NumPy array")

    # Drop rows with NaN or inf values
    original_shape = pct_change.shape
    pct_change = pct_change[~np.isnan(pct_change).any(axis=1)]
    pct_change = pct_change[~np.isinf(pct_change).any(axis=1)]
    print(f"Dropped {original_shape[0] - pct_change.shape[0]} rows with NaN or inf values")

    cointegrated_pairs = []

    # Iterate over each pair of assets
    for i in range(pct_change.shape[1]):
        for j in range(i + 1, pct_change.shape[1]):
            asset1 = pct_change[:, i]
            asset2 = pct_change[:, j]

            # Test for cointegration
            score, pvalue, _ = coint(asset1, asset2)

            # Check if the pair is cointegrated
            if pvalue < 0.05:
                cointegrated_pairs.append((labels[i], labels[j], pvalue))

    if not cointegrated_pairs:
        print("No cointegrated pairs found.")
    else:
        print("Cointegrated pairs:")
        for pair in cointegrated_pairs:
            print(f"{pair[0]} and {pair[1]} (p-value: {pair[2]:.16f})")

    return cointegrated_pairs


file_path = 'crypto.csv'
data = load_data(file_path)
train_data, _ = split_data(data, split_ratio=0.5)
labels, pct_change = calculate_daily_returns(train_data)
cointegrated_pairs = find_cointegrated_pairs(labels, pct_change)
#print(p_value_df)

# OU Process

In [109]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from scipy.optimize import minimize

def find_optimal_hedge_ratio(asset1, asset2, max_iter=1000):
    """
    Find the hedge ratio that minimizes the mean-reverting speed of the spread between two assets.

    Args:
        asset1 (numpy.ndarray): The first asset's time series data.
        asset2 (numpy.ndarray): The second asset's time series data.
        max_iter (int): The maximum number of iterations for the optimization algorithm.

    Returns:
        float: The optimal hedge ratio.
        float: The mean-reverting speed of the spread with the optimal hedge ratio.
    """
    def objective_function(hedge_ratio):
        spread = asset1 - hedge_ratio * asset2
        adf_stat, _, _, _, _, _ = adfuller(spread)
        return -adf_stat  # Minimize the negative ADF statistic (maximize the ADF statistic)

    initial_guess = 1.0  # Initial guess for the hedge ratio
    result = minimize(objective_function, initial_guess, method='Nelder-Mead', options={'maxiter': max_iter})
    optimal_hedge_ratio = result.x[0]
    spread = asset1 - optimal_hedge_ratio * asset2
    adf_stat, _, _, _, _, _ = adfuller(spread)
    mean_reverting_speed = -adf_stat

    return optimal_hedge_ratio, mean_reverting_speed

def find_optimal_hedge_ratios(data, labels):
    """
    Find the optimal hedge ratios and mean-reverting speeds for all pairs of assets.

    Args:
        data (pandas.DataFrame): A DataFrame containing the asset data.
        labels (list): A list of asset labels.

    Returns:
        pandas.DataFrame: A DataFrame containing the optimal hedge ratios and mean-reverting speeds for each pair of assets.
    """
    n = len(labels)
    results = []

    for i in range(n):
        for j in range(i + 1, n):
            asset1 = data.iloc[:, i].values
            asset2 = data.iloc[:, j].values

            # Check for inf or NaN values
            if np.isnan(asset1).any() or np.isnan(asset2).any() or np.isinf(asset1).any() or np.isinf(asset2).any():
                continue

            try:
                optimal_hedge_ratio, mean_reverting_speed = find_optimal_hedge_ratio(asset1, asset2)
                pair_label = f"{labels[i]} - {labels[j]}"
                result = {
                    'Pair': pair_label,
                    'Optimal Hedge Ratio': optimal_hedge_ratio,
                    'Mean-Reverting Speed': mean_reverting_speed
                }
                results.append(result)
            except (ValueError, np.linalg.LinAlgError):
                # Ignore pairs with errors or NaNs
                continue

    return pd.DataFrame(results)




file_path = 'crypto.csv'
data = load_data(file_path)
train_data, _ = split_data(data, split_ratio=0.5)
labels, _ = calculate_daily_returns(train_data)
#cointegrated_pairs = find_cointegrated_pairs(labels, pct_change)
#print(p_value_df)
hedge_ratios = find_optimal_hedge_ratios(train_data, labels)
print(hedge_ratios)


            Pair  Optimal Hedge Ratio  Mean-Reverting Speed
0      BTC - ETH        -4.193887e+01              0.148451
1     BTC - DOGE         8.539004e+01              0.780816
2     BTC - ADA         -2.059356e+03              0.733710
3     BTC - LINK         2.229229e+02              0.724089
4      BTC - TRX        -3.763027e+04              0.757796
5      BTC - UNI         8.575059e+01              0.725373
6    BTC - MATIC        -5.403157e+04             -0.625676
7     ETH - DOGE         3.878019e+03             -1.476407
8     ETH - ADA         -1.458988e+02             -0.241431
9     ETH - LINK         6.142500e+01             -2.564072
10     ETH - TRX         8.652258e+03             -0.435223
11     ETH - UNI         4.188770e+01             -0.090088
12   ETH - MATIC        -2.426916e+03             -0.837723
13   DOGE - ADA          1.196409e+11              1.337782
14   DOGE - LINK         8.417969e-02              1.525924
15    DOGE - TRX        -3.603906e+00   

In [4]:
# Calculate spreads and z-scores
def calculate_spread_and_zscore(pairs, data):
    spread_results = []
    zscore_results = []
    optimal_entries = []
    optimal_exits = []

    for pair in pairs:
        asset1_name, asset2_name, pvalue = pair
        if asset1_name not in data.columns or asset2_name not in data.columns:
            raise ValueError(f"Asset names {asset1_name} or {asset2_name} not found in data columns.")

        asset1 = data[asset1_name]
        asset2 = data[asset2_name]

        asset1, asset2 = asset1.align(asset2, join='inner')
        combined = pd.DataFrame({'asset1': asset1.ffill(), 'asset2': asset2.ffill()}).dropna()

        if combined.empty:
            continue

        asset1_cleaned = combined['asset1']
        asset2_cleaned = combined['asset2']

        spread = asset1_cleaned - asset2_cleaned
        spread_mean = spread.mean()
        spread_std = spread.std()

        optimal_entry = (spread_std * 2.0)
        optimal_exit = (spread_std * 0.5)

        if spread_std == 0:
            zscore = pd.Series([0] * len(spread), index=spread.index)
        else:
            zscore = (spread - spread_mean) / spread_std

        spread_results.append(spread)
        zscore_results.append(zscore)
        optimal_entries.append(optimal_entry)
        optimal_exits.append(optimal_exit)

    return spread_results, zscore_results, optimal_entries, optimal_exits